# Emily Ai Model Training
Depression Detection Model

In [1]:
# All required imports
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
# from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Preprocessing the dataset 

In [2]:
def preprocess(X_train, X_test):
    """
    Convert from float64 to float32 and normalize normalize to decibels
    relative to full scale (dBFS) for the 4 sec clip.
    """
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train])
    X_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test])
    return X_train, X_test

# Train Test divide and Image Resizing

In [3]:
def prep_train_test(X_train, y_train, X_test, y_test, nb_classes):
    """
    Prep samples ands labels for Keras input by noramalzing and converting
    labels to a categorical representation.
    """
    print('Train on {} samples, validate on {}'.format(X_train.shape[0],
                                                       X_test.shape[0]))

    # normalize to dBfS
    X_train, X_test = preprocess(X_train, X_test)

    # Convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    return X_train, X_test, Y_train, Y_test

def keras_img_prep(X_train, X_test, img_dep, img_rows, img_cols):
    """
    Reshape feature matrices for Keras' expexcted input dimensions.
    For 'th' (Theano) dim_order, the model expects dimensions:
    (# channels, # images, # rows, # cols).
    """
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    return X_train, X_test, input_shape



# Model Defination

In [4]:
def cnn(X_train, y_train, X_test, y_test, batch_size,
        nb_classes, epochs, input_shape):
  weights = {
          0: np.random.rand(),
          1: np.random.rand()
        }
  # path = r"/content/drive/MyDrive/Emily"
  checkpoint_filepath = r'./weights-{epoch:02d}-acc{val_accuracy:.4f}-loss{val_loss:.4f}.h5'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
  model = Sequential()
  # model.add(Conv2D(32, (5,5), activation='relu', input_shape=input_shape))
  # model.add(MaxPooling2D((4, 4), strides=4))
  # model.add(Conv2D(32, (3, 3), activation='relu'))
  # model.add(MaxPooling2D((1, 3), strides=(1,3)))
  # model.add(Flatten())
  # model.add(Dense(128, activation='linear'))
  # model.add(Dropout(0.6))
  # model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.8))
  # model.add(Dense(2, activation='sigmoid'))
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.1))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)))
  model.add(MaxPooling2D((2, 2)))
  # model.add(Conv2D(128, (3, 3), activation='relu'))
  # model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(2, activation='sigmoid'))
  model.compile(optimizer=tf.keras.optimizers.Adagrad(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

  history = model.fit(X_train,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1, 
                        validation_data=(X_test, y_test),
                        shuffle=True,
                        callbacks=[model_checkpoint_callback],
                        # validation_split=0.1
                        class_weight={0:np.random.rand(),1:np.random.rand()}, 
                        )
  return model, history




# Main Runner Code


In [5]:
# Loading Dataset
# path = r"/content/drive/MyDrive/Emily"

# Loading Dataset
X_train = np.load(
        r'./train_samples_8sec.npz')
y_train = np.load(
        './train_labels_8sec.npz')
X_test = np.load(
        './test_samples_8sec.npz')
y_test = np.load(
    './test_labels_8sec.npz')

X_train, y_train, X_test, y_test = \
        X_train['arr_0'], y_train['arr_0'], X_test['arr_0'], y_test['arr_0']




In [ ]:
# Hyper Parameters
batch_size = 32
nb_classes = 2
epochs = 100
# Train Test Size Update
X_train, X_test, y_train, y_test = prep_train_test(X_train, y_train,
                                                       X_test, y_test,
                                                       nb_classes=nb_classes)

# 513x125x1 for spectrogram with crop size of 125 pixels
img_rows, img_cols, img_depth = X_train.shape[1], X_train.shape[2], 1

# Spectogram Image Processing
X_train, X_test, input_shape = keras_img_prep(X_train, X_test, img_depth,
                                                  img_rows, img_cols)

# print(y_test)
# print(y_train)

# Loaing and Fitting The model
model, history = cnn(X_train, y_train, X_test, y_test, batch_size,
                         nb_classes, epochs, input_shape)


In [ ]:
score_train = model.evaluate(X_train, y_train, verbose=1)
print('Train accuracy:', score_train[1])
score_test = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', score_test[1])

72/72 [==============================] - 3s 36ms/step - loss: 0.9373 - accuracy: 0.5000
Train accuracy: 0.5
26/26 [==============================] - 1s 35ms/step - loss: 0.9381 - accuracy: 0.5000
Test accuracy: 0.5
